In [1]:
# Use autoplot 207's code for this app
import sys
import datetime
sys.path.insert(0, "/opt/iem/htdocs/plotting/auto/scripts200/")
from p207 import load_data, compute_grid_bounds, add_zeros, do_analysis, USEME
from matplotlib.patches import Rectangle
from shapely.geometry import Point
from pyiem.plot import MapPlot, nwssnow, get_cmap
from pyiem.nws.vtec import NWS_COLORS
from pyiem.util import get_dbconn, utc, get_sqlalchemy_conn
from pyiem.reference import EPSG
from pyproj import Transformer
import numpy as np
import pandas as pd
import geopandas as gpd

In [2]:
STORM_NUMBER = 10
TITLE = "14-15 February 2024"
SUBTITLE = "8 AM 15 February 2024"
SETPOINT_LOCS = {}
sts = utc(2024, 2, 15, 6)
ets = utc(2024, 2, 15, 18)
# Get available data
ctx = {'coop': 'yes', 't': 'state', 'sz': 30, 'z': 'yes', 'f': 'linear', 'v': 'snow', 'wfo': 'DMX'}
df = load_data(ctx, sts, ets)
df = df[~df['nwsli'].isin(['DSXI4', 'DMX'])]
# figure out our grid bounds
ctx["bnds2163"] = compute_grid_bounds(ctx, 'IA')
# add zeros and QC
df = add_zeros(df, ctx)

In [3]:
def overlay_ice(mp):
    """Add plotted ice storm."""
    with get_sqlalchemy_conn("postgis") as conn:
        df = pd.read_sql(
        """
        SELECT st_x(geom) as lon, st_y(geom) as lat, magnitude from lsrs WHERE
        typetext in ('ICE STORM', 'FREEZING RAIN') and magnitude > 0
        and valid > %s and valid < %s and state = 'IA'
        """,
        conn,
        params=(sts - datetime.timedelta(days=1), ets)
        )
    print(df[df["state"] == 'IA'])
    mp.plot_values(df.lon.values, df.lat.values, df.magnitude.values, fmt="%.2f", labelbuffer=1, color='purple')

def workflow(ctx, df, isfinal=False, lower=0, upper=2):
    # do gridding
    df2 = df[df[USEME]]
    lons, lats, vals = do_analysis(df2, ctx)
    mp = MapPlot(
        sector="state",
        state=ctx["csector"],
        axisbg="white",
        title="%s - IEM Snowfall Total Analysis" % (TITLE, ),
        subtitle=("Snowfall totals till %s from NWS COOP, LSR, CoCoRaHS Reports; "
                  "IEM 2022-2023 Winter Storm #%s") % (SUBTITLE, STORM_NUMBER),
        twitter=True,
    )
    cmap = nwssnow()
    # cmap = get_cmap("Greens")
    ramp = [0.1, 1, 2, 3, 4, 6, 8, 12, 18, 24, 30, 36]
    # ramp = [0.1, 1, 2, 3, 4]
    mp.contourf(
        lons, lats, vals, np.array(ramp), cmap=cmap, clip_on=True)
    df_useme_plot = df2[(df2['val']>=lower)&(df2['val'] < upper)]
    print(df[df["state"] == 'IA'])
    mp.drawcounties()
    # overlay_ice(mp)
    if isfinal:
        mp.drawcities()
    else:
        mp.plot_values(df_useme_plot['lon'], df_useme_plot['lat'], df_useme_plot['val'].values,
            '%s', labels=df_useme_plot['nwsli'].values,
            textsize=10, labeltextsize=10, labelbuffer=1)
    return mp

In [4]:
def add_setpoints(setpoints):
    """Manual things."""
    for sp, val in setpoints:
        df.at[10000 + sp, "geo"] = Point(SETPOINT_LOCS[sp][0], SETPOINT_LOCS[sp][1])
        df.at[10000 + sp, "val"] = val
        df.at[10000 + sp, USEME] = True
        df.at[10000 + sp, "plotme"] = True

def draw_setpoints(mp):
    """Add some points where manual obs could be inserted."""
    xlim = mp.panels[0].ax.get_xlim()
    ylim = mp.panels[0].ax.set_ylim()
    sz = ctx["sz"] * 1000.
    i = 0
    trans = Transformer.from_proj(mp.panels[0].crs, 2163, always_xy=True)
    for y in np.arange(ylim[0] + sz/2, ylim[1], sz):
        for x in np.arange(xlim[0] + sz/2, xlim[1], sz):
            mp.panels[0].ax.text(x, y, f"{i}", ha="center", va="center")
            # Need to store the x, y in 2163, which is what p207 uses :/
            (xx, yy) = trans.transform(x, y)
            SETPOINT_LOCS[i] = [xx, yy]
            i += 1

def plotsqw(mp):
    with get_sqlalchemy_conn("postgis") as conn:
        gdf = gpd.read_postgis(
            f"SELECT geom from sbw_{sts.year} w WHERE w.phenomena = 'SQ' and w.issue > %s and w.issue < %s",
            conn,
            params=(sts, ets)
        )
    gdf.to_crs(mp.panels[0].crs).plot(
        ax=mp.panels[0].ax,
        aspect=None,
        edgecolor=NWS_COLORS["SQ.W"],
        facecolor="None",
        zorder=1000,
        linewidth=2,
    )
    p0 = Rectangle((0, 0), 1, 1, ec=NWS_COLORS["SQ.W"], fc="None")
    mp.panels[0].ax.legend((p0, ), ("Snow Squall Warning", ), loc=1).set_zorder(1000)

In [5]:
df

,state,wfo,val,lon,lat,geo,used_for_analysis,nwsli,plotme,source,xcell,ycell
0,CA,STO,17.0,-120.510000,39.410000,POINT (-1741894.731 -403956.599),True,0,True,LSR,-66,4
1,CA,STO,16.0,-120.350000,39.330000,POINT (-1730709.302 -416035.143),True,1,True,LSR,-66,4
2,CA,STO,16.0,-120.350000,39.340000,POINT (-1730452.017 -414946.742),True,2,True,LSR,-66,4
3,CA,STO,14.0,-120.680000,39.620000,POINT (-1750454.249 -377505.894),True,3,True,LSR,-67,5
4,WA,PQR,14.0,-121.570000,45.730000,POINT (-1649311.881 304038.109),True,4,True,LSR,-63,28
...,...,...,...,...,...,...,...,...,...,...,...,...
2685,ME,GYX,0.0,-70.650000,44.966700,POINT (2245713.424 412089.110),True,RANM1,True,COOP,66,31
2686,ME,GYX,0.0,-70.672200,44.989400,POINT (2243207.784 413862.458),True,RGNM1,True,COOP,66,31
2687,ME,GYX,0.0,-70.256100,44.282200,POINT (2301597.993 351004.455),True,TURM1,True,COOP,68,29
2688,Z,NaN,0.0,-92.728043,39.871226,POINT (620227.691 -542858.648),True,Z1,False,NaN,12,0


In [9]:
def main():
    setpoints = [
    ]
    if setpoints:
        add_setpoints(setpoints)
    cull = [
        'WMTI4',
        'ESTI4',
    ]
    if cull:
        df.loc[df['nwsli'].isin(cull), USEME] = False
    hardcode = [
    ]
    for nwsli, val in hardcode:
        df.loc[df['nwsli'] == nwsli, 'val'] = val

    ctx['csector'] = "IA"
    mp = workflow(ctx, df, isfinal=True, lower=0, upper=4)
    # draw_setpoints(mp)

    #plotsqw(mp)
    res = mp.postprocess(filename='240216.png')
    mp.close()

main()

     state  wfo  val      lon      lat                             geo  \
587     IA  FSD  2.0 -95.1800  43.4400  POINT (388911.046 -161963.386)   
636     IA  FSD  1.9 -95.3000  43.4100  POINT (379431.588 -165860.078)   
639     IA  FSD  1.8 -95.1500  43.3300  POINT (392044.482 -174034.418)   
741     IA  ARX  1.0 -91.7600  43.1500  POINT (667135.511 -171996.102)   
797     IA  ARX  0.5 -92.8100  43.2800  POINT (581166.789 -165623.293)   
...    ...  ...  ...      ...      ...                             ...   
2414    IA  OAX  0.0 -95.3906  41.3133  POINT (384925.117 -399093.372)   
2415    IA  OAX  0.0 -95.5667  40.8667  POINT (372813.096 -449482.336)   
2416    IA  OAX  0.0 -95.2333  41.0000  POINT (399987.787 -433122.842)   
2417    IA  OAX  0.0 -95.3500  40.7833  POINT (391515.463 -457736.398)   
2418    IA  OAX  0.0 -95.6811  41.3894  POINT (360265.235 -391959.396)   

      used_for_analysis  nwsli  plotme source  xcell  ycell  
587               False    587   False    LSR    